In [1]:
import re, string,os
from glob import glob as gb
import pandas as pd
from tqdm import tqdm
from collections import Counter
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
from functions import *
from gensim.models import KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.parsing.preprocessing import strip_tags
from gensim.utils import simple_preprocess
from scipy import spatial
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy import spatial
import statistics
import umap
from sklearn.cluster import dbscan
from sklearn.mixture import GaussianMixture
plotting.style_('tab20',16)

In [ ]:
data = data_loader.load_month('gb','2020-03','2020-04')
data['text'] = [utils.preprocess(str(x)) for x in data['lemmatized']]

In [14]:
data = data.drop(["lemmatized",'posner','subcorpus','speaker_name','speaker_gender'],axis=1)

In [17]:
# Measuring Distance between Speech Elements using Doc2Vec

def train_doc2vec(data,language):
    texts = data['text']
    texts = [simple_preprocess(strip_tags(str(doc)), deacc=True) for doc in texts]
    texts = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
    model = Doc2Vec(documents=texts,
                 vector_size=300,
                 min_count=25, window=15,
                 sample=1e-5,
                 negative=5,
                 hs=1,
                 epochs=75,
                 dm=0,
                 dbow_words=1,
                 workers=12)
    model.save(f"/home/ruben/Documents/GitHub/ParlaMintCase/results/models/doc2vec-{language}-e75.model")
    data.to_csv(f"/home/ruben/Documents/GitHub/ParlaMintCase/results/models/doc2vec-{language}-e75-sourcedata.csv",sep='\t',index=False)
    return data,model

In [4]:
# data,m = train_doc2vec(data,'gb')
m = Doc2Vec.load(f"/home/ruben/Documents/GitHub/ParlaMintCase/results/models/doc2vec-gb-e75.model")

In [5]:
## Functions
def umap_clustering(vectors):
    umap_model = umap.UMAP(n_neighbors=75,
                           n_components=5,
                           metric='cosine').fit(vectors) #model.docvecs.vectors_docs
    return umap_model

def gmm_clustering(umap_emb,n_clusters):
    gmm = GaussianMixture(n_components=n_clusters,random_state=17)
    gmm.fit(umap_emb)
    cluster_gmm = gmm.predict_proba(umap_emb)
    return cluster_gmm

def get_topic_vectors(doc2vec_model, gmm_model, num_clusters):
    topic_vectors = np.vstack([doc2vec_model.docvecs.vectors_docs[[c for c,i in enumerate(gmm_model) if np.argmax(i) == x]].mean(axis=0)
                               for x in range(num_clusters)])
    return topic_vectors

def get_topic_words(doc2vec_model,topic_vectors_):
    topic_words_ = []
    topic_word_scores = []

    for tv in topic_vectors_:
        sim_words = doc2vec_model.wv.most_similar(positive=[tv], topn=50)
        topic_words_.append([word[0] for word in sim_words])
        topic_word_scores.append([round(word[1], 4) for word in sim_words])

    topic_words_ = np.array(topic_words_)
    topic_word_scores = np.array(topic_word_scores)
    return topic_words_,topic_word_scores

In [6]:
umap_emb = umap_clustering(m.docvecs.vectors_docs)

In [9]:
gmm_emb = gmm_clustering(umap_emb.embedding_,25)

In [10]:
topic_vectors = get_topic_vectors(m,gmm_emb,25)

In [12]:
topic_words_,topic_word_scores = get_topic_words(m,topic_vectors)

In [14]:
pd.DataFrame(topic_words_)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,mccoll,lugano,noble,irretrievably,lord,irretrievable,divorce,broadcasting,bbc,hague,...,msy,declaration,counselling,lady,agreement,think,hansard,withdraw,kennedy,divergence
1,supermarket,worker,care,staff,volunteer,frontline,food,vulnerable,pharmacist,dementia,...,ensure,panic,cough,medication,chronic,work,worried,ppe,retailer,counsellor
2,education,school,teacher,pupil,student,ofsted,educational,gcs,exam,fe,...,training,early,class,autism,special,mobility,breakfast,disability,graduate,nursery
3,rent,eviction,evict,landlord,marmot,cladding,expectancy,poorest,homelessness,renter,...,budget,austerity,deprive,insecure,living,neighbourhood,inflict,earnings,defer,shelter
4,ceramic,carbon,export,tariff,us,import,renewable,emission,trade,decarbonise,...,steel,animal,battery,lithium,green,oil,cluster,strategy,coal,aquaculture
5,armed,veteran,personnel,civilian,force,gang,military,afghanistan,ministry,army,...,burnley,parole,slavery,war,investigation,policing,tank,integrate,superb,justice
6,peru,strand,embassy,consular,flight,fco,foreign,abroad,repatriation,airline,...,dual,turkey,phone,resort,overseas,iran,route,advise,tourist,office
7,lugano,parole,recess,mccoll,irretrievably,morse,ciga,elgin,indecent,irretrievable,...,curve,respondent,sahara,davis,broadcasting,hotline,committees,orientation,ratify,dissolution
8,committee,parliament,house,committees,recess,motion,speaker,grocott,peerage,order,...,expedient,opposition,party,delegated,constitutional,shall,peer,whip,respondent,wednesday
9,muslims,muslim,religion,indian,india,violence,protest,police,crime,cremation,...,envoy,survivor,faith,woman,prayer,incident,allegation,organ,against,lesbian
